# wPLI

In [53]:
# imports
from pathlib import Path
import re

import numpy as np
import pandas as pd
from scipy.io import loadmat

import mne

## Prepare raw data for MATLAB

In [ ]:
target_ch = ['F1', 'F2', 'F3', 'F4', 'F5', 'F6',
             'P1', 'P2', 'P3', 'P4', 'P5', 'P6']

for file_dir in sorted(Path('data/clean_data/').glob('*.fif')):
    subject, task = re.search('(.*)_ses-01_task-(.*)_proc-clean_epo', file_dir.stem).groups()
    
    # open data
    epochs = mne.read_epochs(file_dir, verbose=0)
    epochs.pick_channels(target_ch)
    
    # epoch to continuous
    ch_names = epochs.ch_names
    data = np.hstack(epochs.get_data())
    info = mne.create_info(ch_names,
                       1000,
                       len(ch_names)*['eeg'],
                       )
    raw = mne.io.RawArray(data, info)
    raw.save(f'data/Data4MATLAB_PLI/{subject}_{task}_raw.fif')

## Open mat file and create datatable for wPLI connectivities

In [21]:
all_wpli = loadmat('all_wpli.mat')

# create a list of files
trial_list = []
for file_dir in sorted(Path('data/clean_data/').glob('*.fif')):
    subject, task = re.search('(.*)_ses-01_task-(.*)_proc-clean_epo', file_dir.stem).groups()
    trial_list.append(subject + '_' + task)

wpli_agg = {}
for i in range(len(trial_list)):
    wpli_agg[trial_list[i]] = all_wpli['all_conns'][0][0][i].mean(1)
    
# connectivity labels
conn_info = all_wpli['all_conns'][0][0][-1][0][0][0]

conn_labels = []
for i in range(len(conn_info)):
    conn_labels.append(conn_info[i][0][0] + '_' + conn_info[i][1][0])

conn_df = pd.DataFrame.from_dict(wpli_agg, columns=conn_labels, orient='index')

numpy.ndarray

In [234]:
# create labels for frontoparietal connections
inter_conn = [i
             for i in conn_labels
             if i[0] + i[3] == 'PP' or i[0] + i[3] == 'FF']
ind = [np.where(np.array(conn_labels) == i)[0][0] for i in inter_conn]
frontal_parietal = np.delete(conn_labels, ind)

frontal_parietal = {
'left-frontal_left-pariental' : ['P3_F3', 'P1_F3', 'P5_F3', 'F1_P3', 'F5_P3', 'P1_F1', 'P5_F1', 'P1_F5', 'P5_F5'],
'left-frontal_right-pariental': ['P4_F3', 'P6_F3', 'P2_F3', 'F1_P4', 'F5_P4', 'P2_F1', 'P6_F1', 'P2_F5', 'P6_F5'],
'right-frontal_left-pariental' : ['P3_F4', 'P5_F4', 'F2_P3', 'P1_F2', 'P5_F2', 'P1_F6', 'P5_F6', 'P1_F4', 'F6_P3'],
'right-frontal_right-pariental' : ['P4_F4', 'P2_F4', 'P6_F4', 'F2_P4', 'F6_P4', 'P2_F2', 'P6_F2', 'P2_F6', 'P6_F6']
}

# aggregate connections between frontal and parietal areas
fp_df = pd.DataFrame()
for k,v in frontal_parietal.items():
    fp_df[k] = conn_df[v].mean(1)

# create labels for connections between traget channels
target_ch = ['P4', 'P3', 'F4', 'F3']
target_connections = pd.DataFrame(columns=target_ch, index=target_ch)
target_connections = target_connections.apply(lambda x: x.index + '_' + x.name)
target_connections = target_connections.values[np.triu_indices(target_connections.shape[0], k=1)]

# join aggregated connectivities and target channel connectivities
df = fp_df.join(conn_df[target_connections])

In [ ]:
# Merge with behavioral data
# open behavioral data and ids map
bh = pd.read_csv('data/behavioral_data/archived/plb_hyp_data.csv', index_col='index')
ids_map = pd.read_excel('docs/ids_map.xlsx', header=1, index_col='behavioral_id')
ids_map = ids_map.drop_duplicates('bids_id')
ids_map = ids_map[['bids_id']]
ids_map['bids_id'] = ids_map['bids_id'].apply(lambda x:str(x).zfill(2))
bh = bh.join(ids_map, how='right')
bh = bh.melt(
    id_vars=['procedure_type_1', 'procedure_type_2', 'procedure_type_3', 'procedure_type_4', 'bids_id',
             'description_type_1', 'description_type_2', 'description_type_3', 'description_type_4'],
    value_vars=['hypnosis_depth_1', 'hypnosis_depth_2', 'hypnosis_depth_3', 'hypnosis_depth_4'])
bh['session'] = bh['variable'].apply(lambda x:x.split('_')[2])
bh['procedure'] = bh.apply(lambda r: r['procedure_type_'+r['session']], axis=1)
bh['description'] = bh.apply(lambda r: r['description_type_'+r['session']], axis=1)
bh = bh[['bids_id', 'value', 'procedure', 'description', 'session']].sort_values(by=['bids_id', 'session']).set_index('bids_id')
bh = bh.rename(columns={'value':'hypnosis_depth'})
bh.reset_index(inplace=True)

df[['bids_id', 'condition']] = df.index.to_series().apply(lambda x:x.split('_')).apply(pd.Series)
df['session'] = df['condition'].apply(lambda x:x[-1])
df.reset_index(drop=True, inplace=True)
df['bids_id'] = df['bids_id'].apply(lambda x:x[4:])
df = pd.merge(bh, df, how='right', on=['session', 'bids_id'], right_index=False)
df = df.sort_values(by=['bids_id', 'session', 'condition']).reset_index(drop=True)
df.insert(1, 'condition', df.pop('condition'))
df.head()

# correct for procedure, description, and hypnosis depth of baseline rows
df.loc[df['condition'] == 'baseline1', ['procedure', 'description']] = 'baseline'
df.loc[df['condition'] == 'baseline2', ['procedure', 'description']] = 'baseline'
df.loc[df['condition'] == 'baseline1', ['hypnosis_depth']] = None
df.loc[df['condition'] == 'baseline2', ['hypnosis_depth']] = None

# save
# df.to_csv('wpli_alpha2.csv', index=0)